In [7]:
from scipy.stats import spearmanr
import pandas as pd
import numpy as np
from glob import glob
import pickle
import os
from  multiprocessing import Pool
from scipy.stats import kendalltau
import matplotlib
import matplotlib.pyplot as plt


# 

files = pd.read_csv('oracle.csv').file.to_list()
libs = [ 'rf', 'xgb', 'lr']

names = ['TAILED', 'NOTNORMAL', 'CONSTFEA', 'PKG', 'SCHEMA', 'CORREDUN', 'MISLABEL2', 
         'CLEAN', 'CLSOVERLAP', 'MISLABCLSOV', 'IMBALANCE']

range_arg = range(0,10)
result_dir = 'Data_oneButall/Results_23/'
thresh_auc = 0.75



# we get the AUCs from metrics pickle!
# later ONLY for the those high AUC ones, we get the model interpretation resutls!


In [3]:
aucs_df = pd.read_pickle('Data_oneButall/metrics_rf_xgb_lr_23.pkl')

print(len(aucs_df))
aucs_df = aucs_df[aucs_df.auc >= thresh_auc]
print(len(aucs_df))

master_auc_df = aucs_df[['auc', 'name', 'file', 'lib']]
master_auc_df.head()

9900
6240


auc    name            file lib
0  0.845805  TAILED  activemq-5.0.0  rf
0  0.851002  TAILED  activemq-5.0.0  rf
0  0.824805  TAILED  activemq-5.0.0  rf
0  0.842101  TAILED  activemq-5.0.0  rf
0  0.899211  TAILED  activemq-5.0.0  rf

# Getting Golden Set

In [4]:
name = 'CLEAN'
file = 'activemq-5.0.0'
lib = 'lr'

df_type = pd.read_pickle(result_dir + file + '_' + name + '_' + lib + '.pkl')        
df_type.head(2)

auc                                               imps lib  \
0  0.809349  {'CountDeclMethodPublic': 0.067, 'CountClassCo...  lr   
0  0.801514  {'CountDeclMethodPublic': 0.055, 'CountClassCo...  lr   

                                              y_pred  \
0  [True, False, False, True, False, False, True,...   
0  [False, False, False, False, True, False, Fals...   

                                              y_test  \
0  0      1
1      0
2      0
3      1
4      0
 ...   
0  0      0
1      0
2      0
3      0
4      0
 ...   

                                               probs  \
0  [1.0, 2.5600405832766293e-18, 8.69222542912194...   
0  [6.159631329057576e-13, 0.011061385425338844, ...   

                                               model  \
0  LogisticRegression(C=49, class_weight='balance...   
0  LogisticRegression(C=25, class_weight='balance...   

                                           filename  
0  Data_oneButall/BootStrapData_23/activemq-5.0.0_0  
0  Data_oneButall/BootStrapData_23/activemq-5.0.0_1

In [8]:
def get_imps_df(name, lib, file):
    '''
    args: gets the name lib and file, 
    returns: the AUC and importances for that specific N, L, F
    '''
    
    df_type = pd.read_pickle(result_dir + file + '_' + name + '_' + lib + '.pkl')        
    ret = df_type[['imps', 'auc']]
    
    return ret



def getImp(file):
    print(file)
    '''
    Args: file name from the oracle to process!
    
    Operation:
        reads the data from master_df to check if the model AUC > 0.75 
            if it is, then reads the actual mini pkl to get the importances.         
        gets all mini pkls, and concatenates them into a golden_df pickle!
    
    Returns:
        golden_df (pd.DataFrame object)
    '''
    
    golden_df = pd.DataFrame()
    
    for lib in libs:  
        for name in names:
            
            this_df = master_auc_df[(master_auc_df.name == name) & (master_auc_df.lib == lib) & (master_auc_df.file == file)]
            
            print(len(this_df), name)
            
            if len(this_df[this_df.auc >= thresh_auc]):
                disk_df =  get_imps_df(name, lib, file)
                df = pd.merge(this_df, disk_df, on = 'auc')                        
                golden_df = pd.concat([golden_df, df]) # saving

    if len(golden_df): # this will be empty if low auc, happens for groovy-1_6_BETA_2
        
        # SAVED
        golden_df = golden_df[golden_df.auc >= 0.75]

        ''' this shouldnt happen unless permutation imp didnt deem any var imp
         which is usually not the case when AUC is high, but can happen '''
        golden_df = golden_df[golden_df.imps != {}]
                
    return golden_df

print(getImp('activemq-5.1.0').head())

activemq-5.1.0
8 TAILED
9 NOTNORMAL
10 CONSTFEA
10 PKG
9 SCHEMA
10 CORREDUN
10 MISLABEL2
10 CLEAN
10 CLSOVERLAP
9 MISLABCLSOV
9 IMBALANCE
5 TAILED
9 NOTNORMAL
9 CONSTFEA
7 PKG
7 SCHEMA
9 CORREDUN
5 MISLABEL2
9 CLEAN
9 CLSOVERLAP
8 MISLABCLSOV
9 IMBALANCE
1 TAILED
7 NOTNORMAL
7 CONSTFEA
9 PKG
8 SCHEMA
9 CORREDUN
8 MISLABEL2
7 CLEAN
9 CLSOVERLAP
9 MISLABCLSOV
7 IMBALANCE
        auc    name            file lib  \
0  0.752688  TAILED  activemq-5.1.0  rf   
1  0.831423  TAILED  activemq-5.1.0  rf   
2  0.781836  TAILED  activemq-5.1.0  rf   
3  0.793211  TAILED  activemq-5.1.0  rf   
4  0.809206  TAILED  activemq-5.1.0  rf   

                                                imps  
0  {'CountInput_Min': 0.006, 'CountClassDerived':...  
1  {'MAJOR_LINE': 0.007, 'CountDeclMethodDefault'...  
2                          {'AvgLineComment': 0.004}  
3                            {'AvgEssential': 0.003}  
4                       {'CountClassCoupled': 0.009}  


In [ ]:
'''  mp fails, dunno y; sequentially takes 1 min only, so wait 4 it'''

files.reverse()
obj = map(getImp, files)
golden_df = pd.concat(obj)

golden_df.head()

In [10]:
golden_df = golden_df.reset_index(drop = True)
golden_df.to_pickle('golden_imps_v4.pkl')
golden_df.head()


auc       name          file lib  \
0  0.789803     TAILED  wicket-1.5.3  rf   
1  0.755171     TAILED  wicket-1.5.3  rf   
2  0.783742  NOTNORMAL  wicket-1.5.3  rf   
3  0.850505  NOTNORMAL  wicket-1.5.3  rf   
4  0.841174  NOTNORMAL  wicket-1.5.3  rf   

                                                imps  
0  {'CountInput_Min': 0.005, 'AvgLineComment': 0....  
1                    {'CountDeclClassMethod': 0.002}  
2  {'CountDeclClass': 0.004, 'MAJOR_LINE': 0.003,...  
3                          {'CountDeclClass': 0.002}  
4                            {'AvgEssential': 0.002}

In [11]:
# converting the imps dict to a df fashion

import multiprocessing
num_processes = multiprocessing.cpu_count()
chunk_size = int(golden_df.shape[0]/num_processes)

chunks = [golden_df.iloc[golden_df.index[i:i + chunk_size]] for i in range(0, golden_df.shape[0], chunk_size)]

def get_df_frm_rows(chunk):
    all_importances = pd.DataFrame()
    for i, row in chunk.iterrows():
        imp_dict = row['imps']
        inner_df = pd.DataFrame.from_records(imp_dict, index = [row['file'] + '_' + row['lib'] + '_' + row['name']])
        all_importances = pd.concat([all_importances,inner_df])
    return all_importances

pool = multiprocessing.Pool(processes=num_processes)
all_importances = pd.concat(pool.map(get_df_frm_rows, chunks))
pool.terminate()
all_importances.head()

AvgLineComment  CountInput_Min  \
wicket-1.5.3_rf_TAILED              0.003           0.005   
wicket-1.5.3_rf_TAILED                NaN             NaN   
wicket-1.5.3_rf_NOTNORMAL             NaN             NaN   
wicket-1.5.3_rf_NOTNORMAL             NaN             NaN   
wicket-1.5.3_rf_NOTNORMAL             NaN             NaN   

                           CountDeclClassMethod  AvgEssential  AvgLineBlank  \
wicket-1.5.3_rf_TAILED                      NaN           NaN           NaN   
wicket-1.5.3_rf_TAILED                    0.002           NaN           NaN   
wicket-1.5.3_rf_NOTNORMAL                   NaN         0.002         0.002   
wicket-1.5.3_rf_NOTNORMAL                   NaN           NaN           NaN   
wicket-1.5.3_rf_NOTNORMAL                   NaN         0.002           NaN   

                           CountDeclClass  MAJOR_LINE  MaxInheritanceTree  \
wicket-1.5.3_rf_TAILED                NaN         NaN                 NaN   
wicket-1.5.3_rf_TAILED                NaN         NaN                 NaN   
wicket-1.5.3_rf_NOTNORMAL           0.004       0.003                 NaN   
wicket-1.5.3_rf_NOTNORMAL           0.002         NaN                 NaN   
wicket-1.5.3_rf_NOTNORMAL             NaN         NaN                 NaN   

                           AvgCyclomaticModified  CountPath_Min  ...  \
wicket-1.5.3_rf_TAILED                       NaN            NaN  ...   
wicket-1.5.3_rf_TAILED                       NaN            NaN  ...   
wicket-1.5.3_rf_NOTNORMAL                    NaN            NaN  ...   
wicket-1.5.3_rf_NOTNORMAL                    NaN            NaN  ...   
wicket-1.5.3_rf_NOTNORMAL                    NaN            NaN  ...   

                           CountLineComment  CountStmt  MINOR_COMMIT  ADEV  \
wicket-1.5.3_rf_TAILED                  NaN        NaN           NaN   NaN   
wicket-1.5.3_rf_TAILED                  NaN        NaN           NaN   NaN   
wicket-1.5.3_rf_NOTNORMAL               NaN        NaN           NaN   NaN   
wicket-1.5.3_rf_NOTNORMAL               NaN        NaN           NaN   NaN   
wicket-1.5.3_rf_NOTNORMAL               NaN        NaN           NaN   NaN   

                           CountSemicolon  MaxNesting_Mean  CountPath_Max  \
wicket-1.5.3_rf_TAILED                NaN              NaN            NaN   
wicket-1.5.3_rf_TAILED                NaN              NaN            NaN   
wicket-1.5.3_rf_NOTNORMAL             NaN              NaN            NaN   
wicket-1.5.3_rf_NOTNORMAL             NaN              NaN            NaN   
wicket-1.5.3_rf_NOTNORMAL             NaN              NaN            NaN   

                           MaxCyclomaticModified  CountOutput_Max  \
wicket-1.5.3_rf_TAILED                       NaN              NaN   
wicket-1.5.3_rf_TAILED                       NaN              NaN   
wicket-1.5.3_rf_NOTNORMAL                    NaN              NaN   
wicket-1.5.3_rf_NOTNORMAL                    NaN              NaN   
wicket-1.5.3_rf_NOTNORMAL                    NaN              NaN   

                           CountLineCode  
wicket-1.5.3_rf_TAILED               NaN  
wicket-1.5.3_rf_TAILED               NaN  
wicket-1.5.3_rf_NOTNORMAL            NaN  
wicket-1.5.3_rf_NOTNORMAL            NaN  
wicket-1.5.3_rf_NOTNORMAL            NaN  

[5 rows x 65 columns]

In [12]:
# NO NULLS because null fixed in the getImp function
all_importances = all_importances.fillna(0)

# Getting rankings for skott knott ranking clustering!
all_importances.to_csv('RQ3_importances_v4.csv')

In [14]:
imps = pd.read_csv('RQ3_importances_v4.csv')
imps.columns

Index(['Unnamed: 0', 'AvgLineComment', 'CountInput_Min',
       'CountDeclClassMethod', 'AvgEssential', 'AvgLineBlank',
       'CountDeclClass', 'MAJOR_LINE', 'MaxInheritanceTree',
       'AvgCyclomaticModified', 'CountPath_Min', 'MaxNesting_Min', 'OWN_LINE',
       'OWN_COMMIT', 'CountDeclMethodPublic', 'CountStmtExe', 'MINOR_LINE',
       'MaxCyclomatic', 'AvgCyclomatic', 'AvgCyclomaticStrict', 'AvgLine',
       'AvgLineCode', 'CountPath_Mean', 'MaxCyclomaticStrict', 'Added_lines',
       'CountDeclInstanceVariable', 'Del_lines', 'CountDeclMethodProtected',
       'CountClassCoupled', 'CountDeclMethodPrivate', 'PercentLackOfCohesion',
       'CountClassBase', 'CountOutput_Min', 'CountClassDerived',
       'CountOutput_Mean', 'CountInput_Mean', 'RatioCommentToCode', 'DDEV',
       'CountLine', 'CountLineCodeExe', 'CountDeclFunction',
       'CountDeclInstanceMethod', 'CountDeclMethodDefault', 'MaxNesting_Max',
       'SumEssential', 'MAJOR_COMMIT', 'SumCyclomatic',
       'SumCyclomat